In [1]:
# add to "../sparse_dgp" sys.path
import os
import sys
current = os.path.dirname(os.path.realpath("__file__"))
parent = os.path.dirname(current)
sys.path.append(parent)

import torch
from dtmgp.utils.sparse_activation.design_class import HyperbolicCrossDesign
from dtmgp.utils.sparse_activation.sparse_grid import SparseGridDesign
from dtmgp.kernels.laplace_kernel import LaplaceProductKernel
from dtmgp.utils.operators.chol_inv import mk_chol_inv, tmk_chol_inv

# Cholesky inverse in one dimension

In [2]:
design_fun = HyperbolicCrossDesign(dyadic_sort=True, return_neighbors=True)
deg = 3
input_bd = [0,1]

dyadic_design = design_fun(deg=deg, input_bd=input_bd)
print(dyadic_design.points)
print(dyadic_design.lefts)
print(dyadic_design.rights)
print(dyadic_design.indices_sort)

markov_kernel = LaplaceProductKernel(lengthscale=1.)
Rinv = mk_chol_inv(dyadic_design=dyadic_design, markov_kernel=markov_kernel, upper=True)

tensor([0.5000, 0.2500, 0.7500, 0.1250, 0.3750, 0.6250, 0.8750])
tensor([  -inf,   -inf, 0.5000,   -inf, 0.2500, 0.5000, 0.7500])
tensor([   inf, 0.5000,    inf, 0.2500, 0.5000, 0.7500,    inf])
tensor([3, 1, 5, 0, 2, 4, 6])


In [3]:
Rinv_dense = Rinv.to_dense()
print(Rinv_dense[:, :5])

tensor([[ 1.0000, -1.2416, -1.2416,  0.0000, -1.4069],
        [ 0.0000,  1.5942,  0.0000, -1.8764, -1.4069],
        [ 0.0000,  0.0000,  1.5942,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  2.1262,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  2.8358],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000]])


In [4]:
ker_input = dyadic_design.points
K_true = markov_kernel(ker_input, ker_input)

R_true = torch.linalg.cholesky(K_true, upper=True)
Rinv_true = torch.linalg.inv(R_true)
print(Rinv_true[:,:5])

tensor([[ 1.0000e+00, -1.2416e+00, -1.2416e+00, -1.6644e-07, -1.4069e+00],
        [ 0.0000e+00,  1.5942e+00,  0.0000e+00, -1.8764e+00, -1.4069e+00],
        [ 0.0000e+00,  0.0000e+00,  1.5942e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  2.1262e+00, -7.6413e-07],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  2.8358e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00]])


In [5]:
print(torch.allclose(Rinv_dense, Rinv_true))
print((Rinv_dense-Rinv_true).norm())

False
tensor(2.5567e-06)


In [6]:
Kinv_true = torch.linalg.inv(K_true)
print(Kinv_true[:5,:5])
Kinv_sp = Rinv_dense @ Rinv_dense.T
print(Kinv_sp[:5,:5])

tensor([[ 8.0416e+00,  2.8354e-07, -2.8235e-07, -5.7131e-07, -3.9896e+00],
        [ 1.8789e-07,  8.0416e+00,  2.2220e-06, -3.9896e+00, -3.9896e+00],
        [ 5.9970e-15,  2.2220e-06,  8.0416e+00, -1.4046e-06, -1.4596e-06],
        [-4.8307e-07, -3.9896e+00, -1.4046e-06,  4.5208e+00, -4.2048e-07],
        [-3.9896e+00, -3.9896e+00, -1.4596e-06, -2.7775e-07,  8.0416e+00]])
tensor([[ 8.0416e+00, -3.5287e-07, -3.5287e-07,  0.0000e+00, -3.9896e+00],
        [-3.5287e-07,  8.0416e+00,  0.0000e+00, -3.9896e+00, -3.9896e+00],
        [-3.5287e-07,  0.0000e+00,  8.0416e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00, -3.9896e+00,  0.0000e+00,  4.5208e+00,  0.0000e+00],
        [-3.9896e+00, -3.9896e+00,  0.0000e+00,  0.0000e+00,  8.0416e+00]])


# Cholesky inverse for sparse grids

In [7]:
# initial setting
d = 2 # dimension
eta = 3 # level
input_bd = [[0,1]]*d # None
design_class = HyperbolicCrossDesign
dyadic_sort = True
indices_sort = True

# generate sparse grid design
sg = SparseGridDesign(d, eta, input_bd=input_bd, design_class=design_class).gen_sg(dyadic_sort=True, return_neighbors=True)
sg.pts_set

tensor([[0.5000, 0.5000],
        [0.5000, 0.2500],
        [0.5000, 0.7500],
        [0.2500, 0.5000],
        [0.7500, 0.5000]])

In [8]:
tensor_markov_kernel = LaplaceProductKernel(lengthscale=1.)
Rinv = tmk_chol_inv(sparse_grid_design=sg, 
                    tensor_markov_kernel=tensor_markov_kernel, 
                    upper = True)
Rinv_dense = Rinv.to_dense()
print(Rinv_dense)

tensor([[ 1.0000, -1.2416, -1.2416, -1.2416, -1.2416],
        [ 0.0000,  1.5942,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  1.5942,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  1.5942,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  1.5942]])


In [9]:
ker_input = sg.pts_set
K_true = tensor_markov_kernel(ker_input, ker_input)
# print(K_true)

R_true = torch.linalg.cholesky(K_true, upper=True)
Rinv_true = torch.linalg.inv(R_true)
print(Rinv_true)

tensor([[ 1.0000, -1.2416, -1.2416, -1.2416, -1.2416],
        [ 0.0000,  1.5942,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  1.5942,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  1.5942,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  1.5942]])


In [10]:
Kinv_true = torch.linalg.inv(K_true)
print(Kinv_true[:5,:5])

Kinv_sp = Rinv_dense @ Rinv_dense.T
print(Kinv_sp[:5,:5])

tensor([[ 7.1660e+00, -1.9793e+00, -1.9793e+00, -1.9793e+00, -1.9793e+00],
        [-1.9793e+00,  2.5415e+00, -1.8029e-08, -1.8029e-08, -1.8029e-08],
        [-1.9793e+00, -1.8029e-08,  2.5415e+00, -1.8029e-08, -1.8029e-08],
        [-1.9793e+00, -1.8029e-08, -1.8029e-08,  2.5415e+00, -1.8029e-08],
        [-1.9793e+00, -1.8029e-08, -1.8029e-08, -1.8029e-08,  2.5415e+00]])
tensor([[ 7.1660, -1.9793, -1.9793, -1.9793, -1.9793],
        [-1.9793,  2.5415,  0.0000,  0.0000,  0.0000],
        [-1.9793,  0.0000,  2.5415,  0.0000,  0.0000],
        [-1.9793,  0.0000,  0.0000,  2.5415,  0.0000],
        [-1.9793,  0.0000,  0.0000,  0.0000,  2.5415]])
